In [7]:
# Device diagnostics (CPU/CUDA/MPS)
import torch

def available_devices():
    devices = []
    if torch.cuda.is_available():
        for idx in range(torch.cuda.device_count()):
            name = torch.cuda.get_device_name(idx)
            cap = torch.cuda.get_device_capability(idx)
            devices.append({"type": "cuda", "index": idx, "name": name, "capability": cap})
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        devices.append({"type": "mps", "index": 0, "name": "Apple MPS"})
    devices.append({"type": "cpu", "index": 0, "name": "CPU"})
    return devices

devices = available_devices()
print("Available devices:")
for d in devices:
    print(d)

x = torch.tensor([1.0])
print("Default tensor device:", x.device)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("Chosen device:", device)


Available devices:
{'type': 'cuda', 'index': 0, 'name': 'NVIDIA A100-SXM4-40GB', 'capability': (8, 0)}
{'type': 'cpu', 'index': 0, 'name': 'CPU'}
Default tensor device: cpu
Chosen device: cuda


In [8]:
# Optional: quick smoke test with torch matrix multiply to verify the device runs
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

a = torch.randn(1024, 1024, device=device)
b = torch.randn(1024, 1024, device=device)
with torch.inference_mode():
    c = a @ b
print("Matmul done on", device, "result shape", c.shape)


Matmul done on cuda result shape torch.Size([1024, 1024])


In [11]:
# Run VLM PoC from notebook (uses GPU if available)
import subprocess
from pathlib import Path
import torch

# Try to locate repo root flexibly
cwd = Path.cwd().resolve()
repo = cwd if (cwd / 'run_poc.py').exists() else cwd / 'vlm_lab'
if not (repo / 'run_poc.py').exists():
    # fallback: assume notebook is inside vlm_lab
    repo = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
python_bin = repo / '.venv/bin/python'
python_exec = python_bin if python_bin.exists() else 'python'
models_file = repo / 'models.yaml'
documents = repo.parent / 'data/input'
output_dir = repo / 'results'
device = 'cuda' if torch.cuda.is_available() else 'auto'

cmd = [
    str(python_exec),
    str(repo / 'run_poc.py'),
    '--documents', str(documents),
    '--models-file', str(models_file),
    '--output-dir', str(output_dir),
    '--max-pages', '1',
    '--device', device,
]

print('Running:', ' '.join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)
print('STDOUT:\n', result.stdout)
print('STDERR:\n', result.stderr)


Running: python /content/run_poc.py --documents /data/input --models-file /content/models.yaml --output-dir /content/results --max-pages 1 --device cuda
STDOUT:
 
STDERR:
 python3: can't open file '/content/run_poc.py': [Errno 2] No such file or directory

